In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import pylab as pl

from psi.controller.calibration.util import load_calibration
from psi.controller.calibration import chirp, tone
from psi.core.enaml.api import load_manifest_from_file

frequencies = [250, 500, 1000, 2000, 4000, 8000, 16000, 32000]

io_file = 'c:/psi-dev/io/pika.enaml'
cal_file = 'c:/psi-dev/io/pika/default.json'

io_manifest = load_manifest_from_file(io_file, 'IOManifest')
io = io_manifest()
audio_engine = io.find('NI_audio')

channels = audio_engine.get_channels(active=False)
load_calibration(cal_file, channels)

mic_channel = audio_engine.get_channel('microphone_channel')
mic_channel.gain = 40

speaker_channel = audio_engine.get_channel('speaker_0')

print(mic_channel.calibration)
print(speaker_channel.calibration)

In [ ]:
tone_calibration = tone.tone_calibration(
    audio_engine, 
    frequencies, 
    ao_channel_name='speaker_0',
    ai_channel_names=['microphone_channel'],
    gain=-30)['microphone_channel']

In [ ]:
chirp_calibration = chirp.chirp_calibration(
    engine=audio_engine,
    ao_channel_name='speaker_0',
    ai_channel_names=['microphone_channel'],
    gain=-30,
    repetitions=8,
)['microphone_channel']

In [ ]:
for f in frequencies:
    t = tone_calibration.get_spl(f, 1)
    c = chirp_calibration.get_spl(f, 1)
    print(f'Tone: {t:0.2f}\tChirp: {c:0.2f}')